In [16]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
#from spellchecker import SpellChecker
import matplotlib.pyplot as plt
from collections import Counter #uso para n-gramas
#from wordcloud import WordCloud
import re # talvez regex vai ajudar na normalizaçao

In [17]:
csv_file_path = 'C:/Users/Administrator/Documents/repos/mestrado/visual_analytics/corpus_completo.csv'

# Lendo o csv como um df
df = pd.read_csv(csv_file_path)

#Criando uma cópia
df_va = df.copy()

In [18]:
df_va.groupby('nota').count()

,file_name,content
nota,,
0,25,25
1,211,211
2,628,628
3,715,715
4,477,477
5,237,237


In [87]:
nltk.download('stopwords')
nltk.download('punkt') # é um tokenizador, importante para nltk.word_tokenize
# nltk.download('rslp') é um stemmer. acho q nao vou usar
nltk.download('punkt_tab') # Precisei que devia fazer download desse pacote pq o punkt nao tava funcionando

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Administrator\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [88]:
# Pré-processamento

# Devo tirar acentos?? Ex: Belém
# Retirar números? Na duvida, retirei.
# Não fiz stemming visto que poderia perder erros ortográficos

#Retirada de sinais gráficos, pontuações e espaços
def clean_cp(text):
    cleaned = text.lower() #Deixando tudo minúsculo
    cleaned = re.sub('[^\w\s]', '', cleaned) # Removendo pontuacao
    #cleaned = re.sub('[0-9]+', '', cleaned) # Removendo números 
    cleaned = re.sub('\d+', '', cleaned) # Removendo números NÃO TÁ FUNCIONANDO AAAAAAAAAAAAAAAAA Começou a funcionar qdo pus o lower como primeiro comando da funçao
    cleaned = re.sub('\s+', ' ', cleaned) # Removendo espaços extras
    cleaned = re.sub('\s+', ' ', cleaned)
    return cleaned.strip() # Removendo tabs

#Retirada de stopwords.. Essa parte deve ser importante na hora de ver as palavras mais frequentes (heatmap)
#def stopwords_cp(text):
  #  stopwords = nltk.corpus.stopwords.words('portuguese')
   # tokenized = nltk.word_tokenize(text, language='portuguese')
   # sem_stopwords = [token for token in tokenized if token not in stopwords]
   # return ' '.join(sem_stopwords)

df_va['content'] = df_va['content'].apply(clean_cp)
#df_va['content'] = df_va['content'].apply(stopwords_cp)

In [89]:
df_va['content'].sample(5)

1662    belém de outubro de senhores prefeitura munici...
1513    atenção prefeitura municipal meu nome é bibian...
1918    belém de outubro de à prefeitura municipal de ...
1855    belém de outubro de prefeitura municipal preza...
76      prezada prefeitura municipal eu estou escreven...
Name: content, dtype: object

In [80]:
df_va.dtypes

file_name    object
content      object
nota          int64
dtype: object

In [107]:
# Tokenizando e retirando stopwords: importante para ver a frequencia das palavras

def tokenized_cp(text):
   stopwords = nltk.corpus.stopwords.words('portuguese') # Carregando as stopwords do português
   tokenized = nltk.word_tokenize(text, language='portuguese') #Transforma o texto em tokens
   text_sem_stopwords = [token for token in tokenized if token not in stopwords] # Deixando somente o que nao é stopword no texto
   return text_sem_stopwords

df_va['tokenized_content'] = df_va['content'].apply(tokenized_cp)

In [115]:
df_va['tokenized_content'].sample(5)

1462    [senhores, prefeitura, municipal, capital, par...
1274    [rua, professor, abeylard, número, apartamento...
541     [excelentísimo, prefeitor, tempo, parar, decui...
627     [exelentisimo, prefeito, belem, pracer, dirigi...
1920    [belém, outubro, senhores, prefeitura, municip...
Name: tokenized_content, dtype: object

In [117]:
df_va.dtypes

file_name            object
content              object
nota                  int64
tokenized_content    object
dtype: object

In [123]:
# Contando o número de tokens sem stopwords para cada texto: usarei isso para o gráfico 2, que demonstra o número mínimo e número máximo de tokens

# Contando tokens sem considerar a nota
df_va['num_tokens'] = df_va['tokenized_content'].apply(len)

# Agrupando de acordo com cada nota
qtde_tokens_nota = df_va.groupby('nota')['num_tokens'].sum()

In [125]:
df_va.tail()

,file_name,content,nota,tokenized_content,num_tokens
2288,20152t4p936n5r.docx,belem de outubro de senhores prefeitura munici...,5,"[belem, outubro, senhores, prefeitura, municip...",100
2289,20152t4p93r.docx,pará de outubro de ao senhor prefeito municipa...,5,"[pará, outubro, senhor, prefeito, municipal, p...",107
2290,20152t4p953n5r.docx,pilas de outubro de prefeitura municipal de be...,5,"[pilas, outubro, prefeitura, municipal, belém,...",94
2291,20152t4p96r.docx,para a prefeitura municipal de um morador de b...,5,"[prefeitura, municipal, morador, belém, senhor...",99
2292,20152t4p997n5r.docx,cidade do belém dia de outubro de exelentíssim...,5,"[cidade, belém, dia, outubro, exelentíssima, p...",145


In [126]:
# Verificando a ocorrência dos tokens

# Criando uma lista só com todos os tokens (?)
all_tokens = [token for tokens in df_va['tokenized_content'] for token in tokens]

# Contando a ocorrencia de cada token
token_counts = Counter(all_tokens)

# Convertendo para um df para ficar melhor de visualizar
# token_counts_df = pd.DataFrame(token_counts.items(), columns=['token', 'count']).sort_values(by='count', ascending=False)

In [127]:
token_counts

Counter({'azulejos': 8132,
         'cidade': 5561,
         'belém': 4062,
         'casarões': 4001,
         'prefeitura': 2903,
         'situação': 2385,
         'municipal': 2249,
         'roubos': 2057,
         'bibiana': 1780,
         'patrimônio': 1725,
         'vandalismo': 1694,
         'cambará': 1646,
         'medidas': 1578,
         'históricos': 1261,
         'proteção': 1239,
         'xix': 1227,
         'maior': 1226,
         'palacete': 1178,
         'problema': 1117,
         'parte': 1093,
         'variedade': 1064,
         'alvo': 1049,
         'século': 1047,
         'valor': 1031,
         'desde': 1023,
         'maria': 1013,
         'mercado': 997,
         'histórico': 990,
         'cultural': 988,
         'valiosos': 957,
         'morador': 947,
         'ano': 931,
         'ter': 902,
         'silva': 888,
         'vitor': 882,
         'atenciosamente': 859,
         'carta': 859,
         'cidades': 851,
         'porque': 847,
   

In [137]:
# Contando tokens de acordo com cada nota NÃO FUNCIONOU

# Define a function to count tokens for each grade
def contagem_tokens_nota(group):
    all_tokens = [token for tokens in group['tokenized_content'] for token in tokens]
    return Counter(all_tokens)

# Group the DataFrame by 'grade' and count tokens in each group
tokens_nota = df_va.groupby('nota').apply(contagem_tokens_nota) # Em breve esse jeito nao vai funcionar

# Convert the result into a DataFrame for easier analysis
tokens_nota_df = (
    tokens_nota
    .apply(lambda x: pd.DataFrame(x.items(), columns=['token', 'count']))
    .reset_index(level=1, drop=True)  # Remove unnecessary index level
    .reset_index()  # Reset index for a clean DataFrame
)

# Print the result
#print(tokens_nota_df)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_17368\3950939695.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tokens_nota = df_va.groupby('nota').apply(contagem_tokens_nota) # Em breve esse jeito nao vai funcionar


IndexError: Too many levels: Index has only 1 level, not 2